# Instant Gratification

## Your first Frontier LLM Project!

Let's build a useful LLM solution - in a matter of minutes.

By the end of this course, you will have built an autonomous Agentic AI solution with 7 agents that collaborate to solve a business problem. All in good time! We will start with something smaller...

Our goal is to code a new kind of Web Browser. Give it a URL, and it will respond with a summary. The Reader's Digest of the internet!!

Before starting, you should have completed the setup for [PC](../SETUP-PC.md) or [Mac](../SETUP-mac.md) and you hopefully launched this jupyter lab from within the project root directory, with your environment activated.

## If you're new to Jupyter Lab

Welcome to the wonderful world of Data Science experimentation! Once you've used Jupyter Lab, you'll wonder how you ever lived without it. Simply click in each "cell" with code in it, such as the cell immediately below this text, and hit Shift+Return to execute that cell. As you wish, you can add a cell with the + button in the toolbar, and print values of variables, or try out variations.  

I've written a notebook called [Guide to Jupyter](Guide%20to%20Jupyter.ipynb) to help you get more familiar with Jupyter Labs, including adding Markdown comments, using `!` to run shell commands, and `tqdm` to show progress.

If you prefer to work in IDEs like VSCode or Pycharm, they both work great with these lab notebooks too.  

## If you'd like to brush up your Python

I've added a notebook called [Intermediate Python](Intermediate%20Python.ipynb) to get you up to speed. But you should give it a miss if you already have a good idea what this code does:    
`yield from {book.get("author") for book in books if book.get("author")}`

## I am here to help

If you have any problems at all, please do reach out.  
I'm available through the platform, or at ed@edwarddonner.com, or at https://www.linkedin.com/in/eddonner/ if you'd like to connect (and I love connecting!)

## More troubleshooting

Please see the [troubleshooting](troubleshooting.ipynb) notebook in this folder to diagnose and fix common problems. At the very end of it is a diagnostics script with some useful debug info.

## If this is old hat!

If you're already comfortable with today's material, please hang in there; you can move swiftly through the first few labs - we will get much more in depth as the weeks progress.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Please read - important note</h2>
            <span style="color:#900;">The way I collaborate with you may be different to other courses you've taken. I prefer not to type code while you watch. Rather, I execute Jupyter Labs, like this, and give you an intuition for what's going on. My suggestion is that you do this with me, either at the same time, or (perhaps better) right afterwards. Add print statements to understand what's going on, and then come up with your own variations. If you have a Github account, use this to showcase your variations. Not only is this essential practice, but it demonstrates your skills to others, including perhaps future clients or employers...</span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business value of these exercises</h2>
            <span style="color:#181;">A final thought. While I've designed these notebooks to be educational, I've also tried to make them enjoyable. We'll do fun things like have LLMs tell jokes and argue with each other. But fundamentally, my goal is to teach skills you can apply in business. I'll explain business implications as we go, and it's worth keeping this in mind: as you build experience with models and techniques, think of ways you could put this into action at work today. Please do contact me if you'd like to discuss more or if you have ideas to bounce off me.</span>
        </td>
    </tr>
</table>

In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [21]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# Connecting to OpenAI

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
# Load environment variables in a file called .env

load_dotenv(dotenv_path='C:\projects\llm_engineering\.env')
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [ ]:
load_dotenv(dotenv_path='C:\projects\llm_engineering\.env')
api_key=os.getenv("OPENAI_API_KEY")

In [3]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the troubleshooting notebook, or try the below line instead:
# openai = OpenAI(api_key="your-key-here-starting-sk-proj-")

In [4]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connect
with me for

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [6]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
system_prompt = "You are an assistant that analyzes the content of a website \
    and provides a short summary, ignoring text that might be navigation related. \
         Respond in markdown. "

In [7]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
def user_prompt_for(website):
    user_prompt=f'You are looking at a website titled {website.title}'
    user_prompt+="\nThe contents of this website is as follows; \
        please provide a short summary of of this website in markdown. \
            If it includes news or announcements, then summarize these too.\n"
    user_prompt+=website.text
    return user_prompt

In [8]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

In [11]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
def messages_for(website):
    return [
        {'role':"system", "content":system_prompt},
        {'role':"user","content":user_prompt_for(website)}
    ]

In [10]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

## Time to bring it together - the API for OpenAI is very simple!

In [13]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [14]:
def summarize(url):
    website=Website(url)
    response=openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner’s website serves as a platform for his interests in coding, LLM (Large Language Model) experimentation, and electronic music production. He introduces himself as a tech enthusiast and the co-founder and CTO of Nebula.io, a company focused on using AI to help individuals discover their potential. He has previously founded the AI startup untapt, which was acquired in 2021.\n\n## Recent Posts\n- **December 21, 2024**: Welcome, SuperDataScientists!\n- **November 13, 2024**: Mastering AI and LLM Engineering – Resources\n- **October 16, 2024**: From Software Engineer to AI Data Scientist – Resources\n- **August 6, 2024**: Outsmart LLM Arena – a battle of diplomacy and deviousness\n\nOverall, the site emphasizes Donner's professional background and his fascination with AI and LLM technologies."

In [15]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [16]:
def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://edwarddonner.com")

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal and professional platform showcasing his interests and contributions in the fields of AI and programming. He is the co-founder and CTO of Nebula.io, an AI-driven company focused on talent discovery. Previously, he founded untapt, which was acquired in 2021. The website reflects his passion for experimenting with large language models (LLMs) and includes a section titled "Outsmart," which introduces an arena for LLMs to compete in diplomacy and strategy.

## Key Announcements and Posts

- **December 21, 2024**: Welcome message to the SuperDataScientists community.
- **November 13, 2024**: Resources for mastering AI and LLM engineering.
- **October 16, 2024**: Guidance for transitioning from a Software Engineer to an AI Data Scientist.
- **August 6, 2024**: Introduction to the Outsmart LLM Arena for competitive AI interactions. 

Through these posts, Ed engages with individuals who share his interests in AI and programming, providing valuable insights and resources.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [18]:
display_summary("https://cnn.com")

# CNN Website Summary

The CNN website offers a wide array of news coverage across various categories including US, World, Politics, Business, Health, and Entertainment, among others. The site features breaking news articles and live updates covering significant global events and topics such as the Ukraine-Russia War and the Israel-Hamas War.

### Recent Key Highlights
- **Tibet Earthquake:** More than 100 people have died due to a powerful quake in Tibet, with CCTV capturing the moment of the disaster.
- **Political Developments:** Rudy Giuliani was held in contempt in a case involving defamation against Georgia election workers. There are also fresh updates regarding the ongoing Trump legal matters and his continuing influence within the GOP.
- **Celebrity News:** Notable mentions include Aubrey Plaza talking about the personal tragedy of losing her husband and celebrity highlights from recent events like the Golden Globes.

### Sections
- The site includes various dedicated sections for weather, sports coverage, health updates, and special features.
- Multimedia options such as videos and live broadcasts are available, enriching the reader's experience with visual content.

The overall focus of the CNN website is to provide timely and comprehensive news coverage, along with engaging multimedia resources for viewers and readers.

In [ ]:
display_summary("https://anthropic.com")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [ ]:
# Step 1: Create your prompts

system_prompt = "something here"
user_prompt = """
    Lots of text
    Can be pasted here
"""

# Step 2: Make the messages list

messages = [] # fill this in

# Step 3: Call OpenAI

response =

# Step 4: print the result

print()

In [19]:
#import libraries

import os
import pandas as pd
from openai import OpenAI
from IPython.display import Markdown, display
from dotenv import load_dotenv

In [33]:
sys_pmt = "You are professional email writer. based on content provided to you summarize and analyze it then \
    you will provide 5 Email subject titles for each email content in a given list of data which will be more suitable for an email based on content." 

In [34]:
data =["I wanted to take a moment to sincerely thank you for your hard work and dedication since joining us in October 2024. Your knowledge of AI/ML has been impressive, and your ability to handle tasks with such efficiency and skill has truly stood out. You've consistently demonstrated a strong commitment to your work, and your contributions have made a positive impact on our team.I appreciate your proactive attitude and willingness to learn. It's been a pleasure having you as part of the team, and I look forward to seeing all the great things you will continue to achieve.",
       "Solar energy is a renewable resource harnessed from the sun's rays. It is one of the cleanest forms of energy, reducing dependence on fossil fuels. Solar panels convert sunlight into electricity through photovoltaic cells. As technology advances, solar energy is becoming more efficient and affordable. It plays a significant role in combating climate change and reducing carbon footprints."
       "Artificial intelligence (AI) refers to machines designed to mimic human intelligence. It encompasses learning, reasoning, and problem-solving capabilities. AI is already used in various sectors like healthcare, finance, and entertainment. The growing field raises ethical questions about job displacement and privacy. However, AI offers immense potential for innovation and improving daily life"
       "Space exploration involves the discovery and exploration of outer space through human or robotic missions. Advancements have allowed humans to land on the Moon, send rovers to Mars, and study distant galaxies. Space agencies like NASA and private companies like SpaceX are pushing boundaries. Future exploration may include colonizing Mars and understanding the universe's origins. The pursuit fuels curiosity and scientific progress."
       "Mental health awareness is about recognizing and understanding mental health issues, reducing stigma, and encouraging open conversations. Conditions such as depression, anxiety, and PTSD affect millions worldwide. Early intervention and proper support can lead to better outcomes for those struggling. Public awareness campaigns help increase empathy and reduce discrimination. Mental health care is becoming a crucial part of overall healthcare systems."
       "Electric vehicles (EVs) are cars powered by electricity rather than gasoline or diesel. They help reduce emissions, decrease air pollution, and offer an environmentally friendly alternative to traditional cars. EV technology has seen rapid improvements in battery life and charging infrastructure. As governments set stricter environmental regulations, EV adoption is expected to grow. Leading brands like Tesla are pushing the market forward."]

In [35]:
def content(data):
        user_pmt = f"The most suitable email subject lines based on your {data} is as follows:\
            display the results in markdown"
        return user_pmt

In [36]:
content(data)

'The most suitable email subject lines based on your ["I wanted to take a moment to sincerely thank you for your hard work and dedication since joining us in October 2024. Your knowledge of AI/ML has been impressive, and your ability to handle tasks with such efficiency and skill has truly stood out. You\'ve consistently demonstrated a strong commitment to your work, and your contributions have made a positive impact on our team.I appreciate your proactive attitude and willingness to learn. It\'s been a pleasure having you as part of the team, and I look forward to seeing all the great things you will continue to achieve.", "Solar energy is a renewable resource harnessed from the sun\'s rays. It is one of the cleanest forms of energy, reducing dependence on fossil fuels. Solar panels convert sunlight into electricity through photovoltaic cells. As technology advances, solar energy is becoming more efficient and affordable. It plays a significant role in combating climate change and red

In [37]:
def user(data):
    return [
        {"role":"system","content":sys_pmt},
        {"role":"user","content":content(data)}   
    ]

In [38]:
user(data)

[{'role': 'system',
  'content': 'You are professional email writer. based on content provided to you summarize and analyze it then     you will provide 5 Email subject titles for each email content in a given list of data which will be more suitable for an email based on content.'},
 {'role': 'user',
  'content': 'The most suitable email subject lines based on your ["I wanted to take a moment to sincerely thank you for your hard work and dedication since joining us in October 2024. Your knowledge of AI/ML has been impressive, and your ability to handle tasks with such efficiency and skill has truly stood out. You\'ve consistently demonstrated a strong commitment to your work, and your contributions have made a positive impact on our team.I appreciate your proactive attitude and willingness to learn. It\'s been a pleasure having you as part of the team, and I look forward to seeing all the great things you will continue to achieve.", "Solar energy is a renewable resource harnessed from

In [39]:
def title(data):
    response=openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=user(data)
    )
    return response.choices[0].message.content

In [40]:
display(Markdown(title(data)))

Here are the summarized email subjects based on the provided content:

### Email Content 1: Thanks and Recognition for Performance
1. **"Thank You for Your Exceptional Contributions!"**
2. **"Celebrating Your Dedication and Hard Work!"**
3. **"Your Impact on Our Team: A Heartfelt Thank You"**
4. **"Acknowledging Your Skills in AI/ML"**
5. **"Looking Forward to Your Continued Success!"**

### Email Content 2: Overview of Solar Energy
1. **"Harnessing the Power of Solar Energy"**
2. **"The Future of Renewable Energy: Solar Power Insights"**
3. **"How Solar Energy is Transforming Our World"**
4. **"Understanding Solar Energy's Role in Climate Change"**
5. **"The Advancements and Benefits of Solar Technology"**

### Email Content 3: Artificial Intelligence Overview
1. **"Exploring the World of Artificial Intelligence"**
2. **"The Impact of AI on Various Sectors"**
3. **"Understanding AI: Opportunities and Ethical Considerations"**
4. **"How AI is Shaping Our Future"**
5. **"Innovations and Challenges in Artificial Intelligence"**

### Email Content 4: Space Exploration Developments
1. **"Pushing Boundaries: The Latest in Space Exploration"**
2. **"Exploring the Universe: Advances in Space Missions"**
3. **"The Future of Space Exploration: Mars and Beyond"**
4. **"Understanding Our Universe: Space Agencies at Work"**
5. **"The Journey of Human Discovery in Outer Space"**

### Email Content 5: Mental Health Awareness
1. **"Shining a Light on Mental Health Awareness"**
2. **"The Importance of Understanding Mental Health Issues"**
3. **"Reducing Stigma: Conversations on Mental Health"**
4. **"Early Intervention: Key to Supporting Mental Wellness"**
5. **"Mental Health Matters: Join the Awareness Movement"**

### Email Content 6: Electric Vehicles Insights
1. **"The Rise of Electric Vehicles: A Green Revolution"**
2. **"EVs: The Future of Eco-Friendly Transportation"**
3. **"Understanding the Benefits of Electric Cars"**
4. **"The Innovations Driving Electric Vehicle Adoption"**
5. **"Reducing Emissions: The Importance of Electric Vehicles"** 

This structured format provides clear email subject lines tailored to each topic, ensuring clarity and relevance.

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

PR instructions courtesy of an AI friend: https://chatgpt.com/share/670145d5-e8a8-8012-8f93-39ee4e248b4c